In [38]:
# %load experiment_embedding_mnist.py
import numpy as np
import os
import csv

from pathlib import Path
from utilities.mnist_exp_util import load_mnist, pre_processing, select_landmarks_mnist_standard, organize_digits
from utilities.embedding import voltage_embedding, multi_dim_scaling
from utilities.data_io import save_config

import matplotlib
import matplotlib.pyplot as plt


def save_experiment(data, target, mds_emb, v_emb, idx_lms, idx_sources, folder):
    print("Save experiment")
    filepath = os.path.join('Results', folder)
    Path(filepath).mkdir(parents=True, exist_ok=True)

    np.savez(os.path.join(filepath, 'embedding_mnist.npz'), mnistdata=data, target=target, mds_emb=mds_emb, v_emb=v_emb)

    with open(os.path.join(filepath, 'idx_lm'), 'w', newline="") as f:
        write = csv.writer(f)
        write.writerows(idx_lms)

    with open(os.path.join(filepath, 'idx_sources'), 'w', newline="") as f:
        write = csv.writer(f)
        write.writerows(idx_sources)

In [39]:
## Main #####################

exp_nr = 1

config = {
    'kernelType': 'radial_scaled',
    'max_iter': 1000,
    'is_Wtilde': False
}

# Variables
bw = 8  # Bandwidth
rhoG = 1.e-3  # Inverse of resistance to ground
rs = 8  # Source radius

num_lm_per_digit = 3
nlm = 10 * num_lm_per_digit
datasize = 1000

folder = f'MnistNlmPerDigit{num_lm_per_digit}_expnr{exp_nr}'
filepath = os.path.join('Results', folder)
Path(filepath).mkdir(parents=True, exist_ok=True)
######################

mnistdata, target = load_mnist()
mnistdata.shape, target.shape

((60000, 784), (60000,))

In [ ]:
mnistdata, target = pre_processing(datasize, mnistdata, target)

In [45]:
# Get indices of all points in x that are distance $r_s$ from lm
source_indices, _ = get_nn_indices(mnistdata, lm.reshape(1, -1), rs) 
source_indices = list(source_indices[0])

# Construct the adjacency matrix W 
matrix = construct_W_matrix(mnistdata, n, bw, rhoG, config)

# Initialize a voltage vector, with source and ground constraints applied
init_voltage = np.zeros(n + 1)
init_voltage = apply_voltage_constraints(init_voltage, source_indices)
# Propagate the voltage to all points in the mnistdataset

NameError: name 'lm' is not defined

In [42]:
voltages.append(propagate_voltage(init_voltage, matrix, config['max_iter'],\
                                  source_indices, config['is_Wtilde'])

SyntaxError: unexpected EOF while parsing (3990951108.py, line 13)

In [26]:
digit_indices = organize_digits(target)
n_mnist = len(mnistdata)
lms, idx_lms = select_landmarks_mnist_standard(mnistdata, digit_indices, num_lm_per_digit)

Loop landmarks: 100%|███████████████████████████████████████████████| 30/30 [00:26<00:00,  1.12it/s]


In [34]:
len(source_indices),v_embedding.shape

(30, (1001, 30))

In [37]:
i=1
v_embedding[source_indices[i],i]

array([0.90057868, 0.90426827, 0.94609223, 0.8946237 , 0.88890686,
       0.90357326, 0.87361643, 0.88594514, 0.89862648, 0.90735034,
       0.89201644, 0.90871701, 0.91509519, 0.87414778, 0.91645123,
       0.91207308, 0.81599129, 0.7799819 , 0.81675102, 0.89640979,
       0.91531974, 0.83179734, 0.87649461, 0.90101484, 0.91839795,
       0.85199324, 0.89851555, 0.90136803, 0.92156701, 0.93237128,
       0.98684211, 0.91931685, 0.91369723, 0.91187382, 0.89066237,
       0.90467589, 0.90675101, 0.9107181 , 0.90041261, 0.89711896,
       0.89246188, 0.91093052, 0.89703745, 0.91224193, 0.90354223,
       0.90879684, 0.81047565, 0.89213729, 0.9054343 , 0.89890256,
       0.86889828, 0.94484914, 0.90733743, 0.8993105 , 0.91034565,
       0.90778491, 0.87498318, 0.92004128, 0.90440198, 0.88230309,
       0.92134374, 0.89044795, 0.91324837, 0.86319374, 0.91931422,
       0.79178443, 0.90157808, 0.90869542, 0.90479696, 0.88580418,
       0.84805775, 0.86502819, 0.88625584, 0.90807178, 0.89998

In [ ]:
source_indices

In [31]:
source_indices[1][:5]

[512, 897, 770, 517, 9]

In [ ]:
mds_embedding = multi_dim_scaling(v_embedding)

save_experiment(mnistdata, target, mds_embedding, v_embedding, idx_lms, source_indices, folder)
save_config(folder, config, bw, rhoG, rs)

v_sort = np.sort(v_embedding, axis=0)
plt.figure()
for i in range(nlm):
    plt.plot(v_sort[:, i], label=f'Digit nr {i}')
plt.legend()
plt.savefig(os.path.join('Results', folder, 'VoltageDecayMnist'))

In [ ]:
# %load utilities/mnist_exp_util.py
from scipy.spatial.distance import cdist
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import os
import csv

def internal_distances(mnistdata):
    distances = cdist(mnistdata, mnistdata)
    plt.figure()
    indices = np.random.choice(np.arange(0, len(mnistdata)), size=100)
    for idx in indices:
        plt.plot(np.sort(distances[idx, :]))

def load_mnist():
    with np.load(os.path.join('Data', 'mnist.npz')) as data:
        x = data['x_train'].reshape(-1, 28*28)
        y = data['y_train']
    return x, y

def pre_processing(datasize, mnistdata, target):
    idx = np.random.choice(np.arange(0, len(mnistdata)), size=datasize)
    mnistdata = mnistdata[idx]
    target = target[idx]
    mnistdata = mnistdata/np.max(mnistdata)
    return mnistdata, target


def organize_digits(target):
    """This function finds the indices of all digit 1,2,3,4 such that we know where each type of digit is
    """
    target_names = np.arange(0, 10)
    digit_indices = []
    for digit_nr in target_names:
        indices = np.where(target == digit_nr)[0]
        digit_indices.append(indices)
    return digit_indices

def select_landmarks_mnist_standard(data, digit_indices, num_lm_per_digit):
    """Standard method for selecting landmarks from mnist. Selects 'num_lm_per_digit'
    randomly for each of the 10 digits"""
    landmarks = []
    idx_lms = []
    for digit_idx in digit_indices:
        indices = np.random.choice(digit_idx, size=min(num_lm_per_digit, len(digit_idx)))
        for idx in indices:
            idx = np.array([idx])
            idx_lms.append(idx)
            landmarks.append(data[idx])
    return landmarks, idx_lms


In [10]:
# %load utilities/embedding.py
import numpy as np
from scipy.spatial.distance import cdist

from utilities.voltage_solver import apply_voltage_constraints, propagate_voltage
from utilities.matrices import construct_W_matrix
from utilities.util import get_nn_indices
from tqdm import tqdm

def voltage_embedding(x, lms, n, bw, rs, rhoG, config):
    """
    Parameters
    ----------
    :param x: Data points n x d numpy array, where n is the number of points, d the dimension
    :param lms: Landmarks m x d numpy array, where m is the number of landmarks, d the dimension
    :param rs: Source radius
    :return:
    """
    voltages = []
    source_indices_l = []
    for lm in tqdm(lms, desc='Loop landmarks'):
        source_indices, _ = get_nn_indices(x, lm.reshape(1, -1), rs)
        source_indices = list(source_indices[0])
        source_indices_l.append(source_indices)

        matrix = construct_W_matrix(x, n, bw, rhoG, config)
        init_voltage = np.zeros(n + 1)
        init_voltage = apply_voltage_constraints(init_voltage, source_indices)
        voltages.append(propagate_voltage(init_voltage, matrix, config['max_iter'],
                                          source_indices, config['is_Wtilde']))
    return np.array(voltages).transpose(), source_indices_l

def multi_dim_scaling(x):
    """ Make multi-dimensional scaling embedding of x
    Parameters
    ----------
    :param x: Coordinates as n x d numpy array, where n is number of training examples and d is the dimension
    :return:
    """
    voltages_centered = x - np.mean(x, axis =0)
    u, sigma, vh = np.linalg.svd(voltages_centered)
    s_temp = np.zeros(len(x))
    s_temp[0:len(sigma)] = sigma[0:len(sigma)]
    sigma = s_temp
    x_mds= np.dot(u, np.diag(sigma))
    return x_mds

# Multi-dim scaling
def distance_matrix(X):
    return np.square(cdist(X.transpose(), X.transpose(), metric='euclidean'))

def gram_from_dist(D, n, s):
    """Calculates gram matrix from distance matrix"""
    I = np.diag(np.ones(n))
    I1 = np.ones(n)
    rhs = (I-np.outer(I1, s))
    lhs = (I-np.outer(s, I1))
    return -1/2*np.dot(rhs, np.dot(D, lhs))

def spectral_cutoff(s, d):
    s_temp = np.zeros(len(s))
    s_temp[0:d] = s[0:d]
    return s_temp

def subsample(X, X_EDM, m):
    index = np.random.choice(X.shape[1], m, replace=False)
    Xw = X[:, index]
    lm_EDM = X_EDM[:, index]
    return Xw, lm_EDM

def center(X):
    c = np.mean(X, axis=1)
    return X - c.reshape(-1, 1), c

def euclidean_distance_matrix(x, s):
    d, n = np.shape(x)
    D = distance_matrix(x)
    G = gram_from_dist(D, n, s)
    u, sigma, uh = np.linalg.svd(G)
    sigma_cut = spectral_cutoff(sigma, d)

    I1 = np.ones(n)
    J = np.diag(np.ones(n)) - np.outer(s, I1)
    x_shifted = np.dot(x, J)
    return np.dot(np.diag(np.sqrt(sigma_cut)), u.transpose()), x_shifted

def orth_procrustes_x_to_edm(x, x_edm, n, m):
    """Implements orthogonal procrustes problem. Rotates x to align with x_edm"""

    x_sub, lm_edm = subsample(x, x_edm, m)

    lm_edm, center_lm_edm = center(lm_edm)
    x_sub, center_x = center(x_sub)

    x_sub_lm_edm = np.dot(x_sub, lm_edm.transpose())
    u, s, vh = np.linalg.svd(x_sub_lm_edm, full_matrices=False)

    R = np.dot(vh.transpose(), u.transpose())
    I1 = np.ones(n)
    x_shift = x - np.outer(center_x, I1)

    return np.dot(R, x_shift) + np.outer(center_lm_edm, I1)


def orth_procrustes_edm_to_x(x, x_edm, n, m):
    """Implements orthogonal procrustes problem. Rotates x_edm to align with x"""

    lm_x, edm_sub = subsample(x, x_edm, m)
    lm_x, center_lm_x = center(lm_x)
    edm_sub, center_edm = center(edm_sub)

    lm_x_edm_sub = np.dot(edm_sub, lm_x.transpose())
    u, s, vh = np.linalg.svd(lm_x_edm_sub, full_matrices=False)

    R = np.dot(vh.transpose(), u.transpose())
    I1 = np.ones(n)
    x_shift = x_edm - np.outer(center_edm, I1)

    return np.dot(R, x_shift) + np.outer(center_lm_x, I1)


In [11]:
tqdm?